In [1]:
import tweepy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob, Word, Blobber
import pandas as pd
import numpy as np
from math import pi
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
import re #regular expressions
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.io import output_notebook, output_file,show
import matplotlib.pyplot as plt

In [2]:
A_Key="V2MKVagJ7FHniRjtZ6hv1nOx3"
A_Key_Secret="Wza1nJO6oSAhUryZ68arQDMvEPh3GbhcpBn9kqPHl30uw9gRw9"
A_Token = "1435660546930475012-fzLwcYBUzeqU4Q29T12mc9JHkn4XgS"
A_Token_Secret = "c7Sl327JDfIZkHw8s3LENFgH77tKcutM9lcG3m59C9Cxh"

In [3]:
auth = tweepy.OAuthHandler(A_Key,A_Key_Secret)
auth.set_access_token(A_Token,A_Token_Secret)
api = tweepy.API(auth)

In [4]:
def get_tweetsdf(topic):
    tweetList = []
    userList = []
    likesList = []
    datetimeList = []
    locationList = []

    cursor = tweepy.Cursor(api.search_tweets,q = topic,lang = "en",tweet_mode = "extended",exclude="retweets").items(150)
    for t in cursor:
        tweetList.append(t.full_text)
        userList.append(t.user.name)
        likesList.append(t.favorite_count)
        locationList.append(t.user.location)
        datetimeList.append(t.created_at)
        
    df =  pd.DataFrame({"User Name":userList,"Tweets":tweetList,"Likes":likesList,
                            "Date Time":datetimeList,"Location":locationList })
    return df

In [5]:
sample = get_tweetsdf("virat")
sample


Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude
Unexpected parameter: exclude


,User Name,Tweets,Likes,Date Time,Location
0,Ashutosh Nevse🎩,@mrsubhani15 Even kohli struggled in early yea...,0,2021-11-25 05:38:10+00:00,"Wai, India"
1,Harsh,@ShivaleekaO One word for 'King Kohli'\n#AskSh...,0,2021-11-25 05:38:05+00:00,
2,Vaibhav yadav,There is no one more committed to Indian crick...,0,2021-11-25 05:37:36+00:00,India
3,Harsh,@ShivaleekaO One word for' Virat Kohli'\n#AskS...,0,2021-11-25 05:37:34+00:00,
4,Kunal,@msdian__fan Yes because of such people \nVira...,1,2021-11-25 05:37:26+00:00,
...,...,...,...,...,...
145,The_Introvert_Official 🇮🇳,@JimmyNeesh Virat says Hi😛🙄\n@imVkohli #tossto...,0,2021-11-25 04:50:09+00:00,"Kerala, India"
146,Atish.,Ajinkya bhau💪🏻\nVirat should win too🙏🏻 or else...,0,2021-11-25 04:50:02+00:00,"Mumbai, India"
147,naqshpa,@sharma9902 @VikrantSolver @Rakeshk72798087 @C...,1,2021-11-25 04:49:22+00:00,india
148,عام پاکستانی,@Cricketracker Babar Azam in place of Virat. V...,0,2021-11-25 04:49:18+00:00,"Islamabad, Pakistan"


In [6]:
def get_polarity_dataframe(df):
        #Section 1
            tempdf = df
            x = tempdf["Tweets"]

            nltk.download('stopwords') #to remove stop words(a,an the)
        
            stop_words=stopwords.words('english') # to get stem of verb(playing->play)
            stemmer=PorterStemmer()

            cleaned_text=[]
            for i in range(len(x)):
                tweet=re.sub('[^a-zA-Z]',' ',x.iloc[i]) #substitute empty string where char is not a-zA-Z
                tweet=tweet.lower().split()

                tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)] #stemming and remove stop words
                tweet=' '.join(tweet)
                cleaned_text.append(tweet)

            tempdf["Clean Tweets"] = cleaned_text

        #Section2
            newdf = sentiment_analysis(tempdf)
            return newdf

In [10]:
pold = get_polarity_dataframe(sample)
pold

[nltk_data] Downloading package stopwords to C:\Users\Anup
[nltk_data]     Jha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,User Name,Tweets,Likes,Date Time,Location,Clean Tweets,Subjectivity,Polarity,Polarity Analysis
0,Ashutosh Nevse🎩,@mrsubhani15 Even kohli struggled in early yea...,0,2021-11-25 05:38:10+00:00,"Wai, India",mrsubhani even kohli struggl earli year virat ...,0.333333,0.250,positive
1,Harsh,@ShivaleekaO One word for 'King Kohli'\n#AskSh...,0,2021-11-25 05:38:05+00:00,,shivaleekao one word king kohli askshivaleeka,0.000000,0.000,neutral
2,Vaibhav yadav,There is no one more committed to Indian crick...,0,2021-11-25 05:37:36+00:00,India,one commit indian cricket dhoni virat kohli say,0.000000,0.000,neutral
3,Harsh,@ShivaleekaO One word for' Virat Kohli'\n#AskS...,0,2021-11-25 05:37:34+00:00,,shivaleekao one word virat kohli askshivaleeka,0.000000,0.000,neutral
4,Kunal,@msdian__fan Yes because of such people \nVira...,1,2021-11-25 05:37:26+00:00,,msdian fan ye peopl virat got unnecessari hate,0.900000,-0.800,negative
...,...,...,...,...,...,...,...,...,...
145,The_Introvert_Official 🇮🇳,@JimmyNeesh Virat says Hi😛🙄\n@imVkohli #tossto...,0,2021-11-25 04:50:09+00:00,"Kerala, India",jimmyneesh virat say hi imvkohli tosstowin ind...,0.000000,0.000,neutral
146,Atish.,Ajinkya bhau💪🏻\nVirat should win too🙏🏻 or else...,0,2021-11-25 04:50:02+00:00,"Mumbai, India",ajinkya bhau virat win els one stop troller in...,0.400000,0.800,positive
147,naqshpa,@sharma9902 @VikrantSolver @Rakeshk72798087 @C...,1,2021-11-25 04:49:22+00:00,india,sharma vikrantsolv rakeshk criccrazyjohn well ...,0.666667,-0.500,negative
148,عام پاکستانی,@Cricketracker Babar Azam in place of Virat. V...,0,2021-11-25 04:49:18+00:00,"Islamabad, Pakistan",cricketrack babar azam place virat virat short...,0.300000,0.000,neutral


In [7]:
def sentiment_analysis(df):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity #subjectivity  =1 (personal opinion) =0(factual point)
        
        #Create a function to get the polarity
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity #polarity lies [-1,1]
        
        #Create two new columns ‘Subjectivity’ & ‘Polarity’
    df["Subjectivity"] =    df["Clean Tweets"].apply(getSubjectivity)
    df["Polarity"] = df["Clean Tweets"].apply(getPolarity)
    def getAnalysis(score):
        if score <= -0.1:
            return "negative"
        elif score > -0.1 and score < 0.1:
            return "neutral"
        else:
            return "positive"
    df["Polarity Analysis"] = df["Polarity"].apply(getAnalysis )
    return df

In [ ]:
def get_polarity_plot(topic):
  #get dataframe from topic
  df1 = get_tweetsdf(topic=topic)

  #get dataframe with polarity columns
  newdf = get_polarity_dataframe(df1)

  posNumber = newdf[newdf["Polarity Analysis"]=="positive"].shape[0]
  negNumber = newdf[newdf["Polarity Analysis"]=="negative"].shape[0]
  neuNumber = newdf[newdf["Polarity Analysis"]=="neutral"].shape[0]

  sentiments = ["Positive","Negative","Neutral"]
  counts = [posNumber, negNumber, neuNumber]
  total = posNumber + negNumber + neuNumber
  print(counts)

In [8]:
text_p = []
for i in sample:
    # print(i.split())
    text_p = ' '.join(i.split())
    print(text_p)
    

User Name
Tweets
Likes
Date Time
Location


In [9]:
word_cloud_p = WordCloud(collocations = False, background_color = 'white').generate(text_p)
plt.imshow(word_cloud_p, interpolation='bilinear')
plt.axis("off")
plt.show()

NameError: name 'WordCloud' is not defined

In [ ]:
proc1 = re.compile('<.*?>')
def proc(text_in):
    text_o = re.sub(proc1, '', text_in) # Removes HTML Tags and other symbols under proc1
    output = re.sub(r'[^\w\s]', '', text_o) #Removes URLs from the text 
    return output.lower()  #Returns the processed text in lower case

In [ ]:
from wordcloud import WordCloud, STOPWORDS
word_cloud_p_processed = WordCloud(collocations = False, background_color = 'white').generate(proc(text_p)) #Stopwords are removed in this line by 3rd argument
plt.imshow(word_cloud_p_processed, interpolation='bilinear')
plt.axis("off")
plt.show()